In [1]:
import pathlib
from import_deps import PyModule, ModuleSet
import pygraphviz

In [2]:
pytrans_module_root = r'C:\Users\carmelo\eth\projects\pytrans-examples\deps\pytrans\pytrans'

def get_imports(pkg_modules, module_path):
    module = pkg_modules.by_path[module_path]
    imports = pkg_modules.get_imports(module, return_fqn=True)
    return list(sorted(imports))


def _imports(module_root):
    """find imports from a python module"""
    base_path = pathlib.Path(module_root)
    pkg_modules = ModuleSet(base_path.glob('**/*.py'))
    imports = {}
    for _, module in pkg_modules.by_name.items():
        importname = '.'.join(module.fqn)
        imports[importname] = get_imports(pkg_modules, module.path)
    return imports




In [10]:
# def module_to_dot(imports, target='pytrans.dot'):
#     graph = pygraphviz.AGraph(strict=False, directed=True)
#     graph.node_attr['color'] = 'lightblue2'
#     graph.node_attr['style'] = 'filled'
#     for source, sinks in imports.items():
#         for sink in sinks:
#             graph.add_edge(source, sink)
#     graph.write(target)

def module_to_dot(imports, target='pytrans.dot'):
    graph = pygraphviz.AGraph(strict=False, directed=True)
    graph.node_attr['style'] = 'filled'
    for source, sinks in imports.items():
        if source.endswith('__init__'):
            color = 'lightyellow'
            source = source[:-9]
        else:
            color = 'lightblue2'
        for sink in sinks:
            graph.add_edge(source, sink)
            node = graph.get_node(source)
            node.attr['color'] = color
    
    graph.write(target)


In [11]:
imports = _imports(pytrans_module_root)

module_to_dot(imports)

In [5]:
imports

{'pytrans.abstract_model': ['pytrans.electrode', 'pytrans.typing'],
 'pytrans.conversion': ['pytrans.ions'],
 'pytrans.electrode': [],
 'pytrans.functions': ['pytrans.conversion', 'pytrans.ions'],
 'pytrans.indexing': ['pytrans.abstract_model', 'pytrans.typing'],
 'pytrans.ions': [],
 'pytrans.objectives': ['pytrans.abstract_model',
  'pytrans.indexing',
  'pytrans.ions'],
 'pytrans.sampling': [],
 'pytrans.solver': ['pytrans.objectives'],
 'pytrans.timer': [],
 'pytrans.typing': [],
 'pytrans.__init__': ['pytrans.abstract_model', 'pytrans.electrode'],
 'pytrans.analysis.analysis': ['pytrans.abstract_model',
  'pytrans.analysis.mode_solver',
  'pytrans.analysis.results',
  'pytrans.ions',
  'pytrans.plotting.plotting',
  'pytrans.plotting.roi',
  'pytrans.timer',
  'pytrans.typing'],
 'pytrans.analysis.analytic_traps': ['pytrans.conversion', 'pytrans.ions'],
 'pytrans.analysis.mode_solver': ['pytrans.abstract_model',
  'pytrans.analysis.results',
  'pytrans.ions',
  'pytrans.typing'],
